Mock up streamlit app

In [46]:
import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import mlflow
import mlflow.sklearn

In [5]:
homedir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
homedir+"/Data"

'c:\\Users\\lay.IFO.000\\Documents\\ifoHackData'

In [8]:
Landprices_raw = pd.read_csv(homedir+"/Data/1 Land Prices/Land_Prices_Neighborhood_Berlin.csv", sep=";")

In [ ]:
Convert Area_Type into numeric

In [31]:
conditions = [
    (Landprices_raw["Area_Types"] == "MixedUseArea_Residential_SpecialUseArea"),
    (Landprices_raw["Area_Types"] == "CommercialBuildingArea_MixedUseArea_Residential_SpecialUseArea"),
    (Landprices_raw["Area_Types"] == "Residential"),
    (Landprices_raw["Area_Types"] == "CommercialBuildingArea_MixedUseArea_Residential"),
    (Landprices_raw["Area_Types"] == "CommercialBuildingArea_Residential"),
    (Landprices_raw["Area_Types"] == "CommercialBuildingArea_Residential_SpecialUseArea"),
    (Landprices_raw["Area_Types"] == "Residential_SpecialUseArea"),
    (Landprices_raw["Area_Types"] == "MixedUseArea"),
    (Landprices_raw["Area_Types"] == "MixedUseArea_Residential"),
]

values = [1,2,3,4,5,6,7,8,9]

Landprices_raw["Area_Types_num"] = np.select(conditions, values)

In [32]:
X = Landprices_raw[["Area_Types_num", "Area_Count"]]
y = Landprices_raw[["Land_Value"]]

In [48]:
rtr = tree.DecisionTreeRegressor()
rtr= rtr.fit(X,y)

rtr.predict([[1,3]])

c:\Users\lay.IFO.000\.conda\envs\ifohack_spatial_py310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([7877.8])

In [33]:
X_train, X_test, y_train, y_test = train_test_split(
    X,y, test_size=0.15, random_state = 42
)

Start Model pipeline to compare different models

In [41]:
def create_pipeline(alpha: int) -> Pipeline:
    return Pipeline(
        steps=[('scalar', StandardScaler()),
               ('model', linear_model.Ridge( 
               ))])


In [54]:
alpha_to_try = [0.1,0.2,0.3,0.4,0.5,0.6]
REGISTERED_MODEL_NAME = "land_price_ridge_regression"  

for alpha in alpha_to_try:
    with mlflow.start_run():
        # build a pipeline with a ridge regression model
        model_pipeline = create_pipeline(alpha=alpha)
        model_pipeline.fit(X_train, y_train)

        # calculaye the mean squared error using the test data
        y_pred = model_pipeline.predict(X=X_test)
        mse = mean_squared_error(
            y_true=y_test, y_pred=y_pred, squared=False, multioutput='uniform_average')

        # log parameters, metrics and the model
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_metric(key="mean_squared_error", value=mse)
        mlflow.sklearn.log_model(
            sk_model=model_pipeline, artifact_path="house_model_forest", registered_model_name=REGISTERED_MODEL_NAME)

        print(
            f"Model saved in run {mlflow.active_run().info.run_uuid}. RMSE={mse}")

Successfully registered model 'land_price_ridge_regression'.
2023/04/29 13:40:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: land_price_ridge_regression, version 1
Created version '1' of model 'land_price_ridge_regression'.


Model saved in run 2c7981f430944d368fce87e13550d07c. RMSE=1775.4096367114314


Registered model 'land_price_ridge_regression' already exists. Creating a new version of this model...
2023/04/29 13:40:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: land_price_ridge_regression, version 2
Created version '2' of model 'land_price_ridge_regression'.


Model saved in run d9a10298d1c348fcadca608d621604c7. RMSE=1775.4096367114314


Registered model 'land_price_ridge_regression' already exists. Creating a new version of this model...
2023/04/29 13:40:19 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: land_price_ridge_regression, version 3
Created version '3' of model 'land_price_ridge_regression'.


...

Registered model 'land_price_ridge_regression' already exists. Creating a new version of this model...
2023/04/29 13:40:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: land_price_ridge_regression, version 5
Created version '5' of model 'land_price_ridge_regression'.


Model saved in run 9a5d749484064becaa07510c3c9214be. RMSE=1775.4096367114314


Registered model 'land_price_ridge_regression' already exists. Creating a new version of this model...
2023/04/29 13:40:31 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: land_price_ridge_regression, version 6


Model saved in run 9e447c1228d7419b8d8022c8f1cd7e72. RMSE=1775.4096367114314


Created version '6' of model 'land_price_ridge_regression'.


In [44]:
!mlflow ui

^C


In [55]:
# only necessary, if you did not stage a model for production using the web interface
client = mlflow.MlflowClient()
client.transition_model_version_stage(
    name=REGISTERED_MODEL_NAME,
    version=5,  # todo: choose the model version, if not already done in the web interface
    stage="Production"
)

<ModelVersion: aliases=[], creation_timestamp=1682768427759, current_stage='Production', description=None, last_updated_timestamp=1682768452068, name='land_price_ridge_regression', run_id='9a5d749484064becaa07510c3c9214be', run_link=None, source='file:///c:/Users/lay.IFO.000/Documents/ifoHack/notebooks/mlruns/0/9a5d749484064becaa07510c3c9214be/artifacts/house_model_forest', status='READY', status_message=None, tags={}, user_id=None, version=5>

Model deployment

In [63]:
mlflow.models.build_docker(
    model_uri=f"models:/{REGISTERED_MODEL_NAME}/Production", name="land_price_model_api", env_manager="local")

2023/04/29 14:16:34 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2023/04/29 14:16:34 INFO mlflow.models.docker_utils: Building docker image with name land_price_model_api
#1 [internal] load build definition from Dockerfile
#1 sha256:22ce8b4f5259bbc45ebd218fa6401dfd2e7a51b1409dfc754a945542c39b9225
#1 DONE 0.0s

#1 [internal] load build definition from Dockerfile
#1 sha256:22ce8b4f5259bbc45ebd218fa6401dfd2e7a51b1409dfc754a945542c39b9225
#1 transferring dockerfile: 2.72kB done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:832c0b7011c84e5c47e6b8fc8af2db5288664933587d3e68346f9de16b236280
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:20.04
#3 sha256:8e67b796a66f85f06793e026943184e32d365c77929e94d2ac98b34a1e1cb30e
#3 DONE 0.6s

#4 [ 1/21] FROM docker.io/library/ubuntu:20.04@sha256:db8bf6f4fb351aa7a26e27ba2686cf35a6a409f65603e59d4c203e58387dc6b3
#4 sha256:1bae5227e407d99860558d8d8b